In [1]:
import os
import pandas as pd
from tqdm.notebook import tqdm

import sys
sys.path.append( '.' )
import importlib

import measures_common
importlib.reload(measures_common)

<module 'measures_common' from 'd:\\Projetos\\TCC\\Part_2.1-Complex_Networks\\measures_common.py'>

In [41]:
file_path = os.path.join('csvs', 'pre_normalized_measures.csv')
df = pd.read_csv(file_path)
df.head(3)

,text,label,betweenness,closeness,eigenvector,katz,pagerank,hubs,authorities,clustering,average_clustering,correlation,transitivity,density
0,autor bestseller cita 5 expressoes indicam int...,fake,0.053793,0.107789,0.054755,0.078508,0.006211,0.006211,0.006211,0.016841,0.016841,-0.041276,0.015000,0.008579
1,juiz df confirma indicios expresidente petista...,fake,0.194226,0.065108,0.076284,0.116096,0.013514,0.013514,0.013514,0.023574,0.023574,-0.082239,0.047619,0.015920
2,senhora 60 anos corre atras eduardo cunha aero...,fake,0.163834,0.137540,0.160614,0.182358,0.033333,0.033333,0.033333,0.013889,0.013889,0.010157,0.050000,0.041379


In [42]:
#Necessary to convert the labels to 0 and 1 for OneR

df['label'] = df['label'].replace({"fake": 0, "true": 1})

In [43]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.svm import OneClassSVM
from sklearn.cluster import KMeans

from mlxtend.classifier import OneRClassifier

NB = GaussianNB()
DT = DecisionTreeClassifier()
RF = RandomForestClassifier()
KNN = KNeighborsClassifier()
SVM = SVC()
MLP = MLPClassifier(max_iter=1000)
OneR = OneRClassifier()

# KM = KMeans(n_clusters=2)
# OC = OneClassSVM(gamma='auto')

# algs = [NB, DT, RF, KNN, SVM, MLP, OC, KM, OneR]
# algs_names = ['NB', 'DT', 'RF', 'KNN', 'SVM', 'MLP', 'OC', 'KM', 'OneR']

algs = [NB, DT, RF, KNN, SVM, MLP, OneR]
algs_names = ['NB', 'DT', 'RF', 'KNN', 'SVM', 'MLP', 'OneR']

In [48]:
from collections import defaultdict
from sklearn.model_selection import cross_val_score

scores = defaultdict(list)
measures = df.columns[2:]
print(measures)

for alg, alg_name in tqdm(zip(algs, algs_names), total=len(algs), position=0, leave=True, desc="All Algorithms"):
    for measure in tqdm(measures, total=len(measures), position=1, leave=True, desc=alg_name):
        scores["measure"].append(measure)
        scores["algorithm"].append(alg_name)
        scores["accuracy"].append(cross_val_score(alg, df[measure].values.reshape(-1, 1), df['label'], cv=5, scoring='accuracy').mean())

Index(['betweenness', 'closeness', 'eigenvector', 'katz', 'pagerank', 'hubs',
       'authorities', 'clustering', 'average_clustering', 'correlation',
       'transitivity', 'density'],
      dtype='object')


All Algorithms:   0%|          | 0/7 [00:00<?, ?it/s]

NB:   0%|          | 0/12 [00:00<?, ?it/s]

DT:   0%|          | 0/12 [00:00<?, ?it/s]

RF:   0%|          | 0/12 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [45]:
df_scores = pd.DataFrame(scores)
df_scores.head(3)

,measure,algorithm,accuracy
0,betweenness,NB,0.509722
1,closeness,NB,0.533611
2,eigenvector,NB,0.507500


In [46]:
df_scores.loc[df_scores.groupby(["measure"])["accuracy"].idxmax()].set_index("measure")\
                                                      .sort_values(by=["accuracy"], ascending=[False])

,algorithm,accuracy
measure,,
closeness,SVM,0.542083
correlation,MLP,0.535417
transitivity,MLP,0.525972
density,DT,0.522639
katz,DT,0.510278
betweenness,NB,0.509722
authorities,SVM,0.508889
hubs,SVM,0.508889
pagerank,SVM,0.508889


In [47]:
df_scores.groupby("algorithm").mean().sort_values(by=["accuracy"], ascending=[False])

,accuracy
algorithm,
SVM,0.511273
NB,0.508171
RF,0.506563
MLP,0.506481
DT,0.505880
KNN,0.504155
OneR,0.503542
